In [1]:
import phoenix as px

session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [2]:
# if you want to view in ipynb
px.active_session().view()

📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


In [3]:
# pip install openinference-instrumentation-dspy opentelemetry-exporter-otlp

from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [4]:
import dspy

lm = dspy.OpenAI()
dspy.configure(lm=lm)

simpleqa = dspy.ChainOfThought("question -> answer")

In [5]:
answer = simpleqa(question="When is last Crusades begin? and Who won?").answer
print(answer)

The last Crusades began in 1270 and ended in 1291. The Mamluks (Muslims) won the last Crusades.


In [21]:
## OpenAI call
# https://platform.openai.com/docs/guides/text-generation/chat-completions-api

from openai import OpenAI
from phoenix.trace.openai import OpenAIInstrumentor

client = OpenAI()
OpenAIInstrumentor().instrument() # this will trace openai call

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

print(response.choices[0].message.content)

The 2020 World Series was played at Globe Life Field in Arlington, Texas.


In [6]:
# See result from pheonix

print(session.url)

http://localhost:6006/


In [26]:
### call한 span의 종류에 따라서 다르게 생성되는거 확인
### .to_csv method를 통해서 export 가능.

df = px.Client().get_spans_dataframe(filter_condition='span_kind=="LLM"') ## 여기에 filter condition 등 이용
# df.to_csv("log.csv")
print(len(df))

df = px.Client().get_spans_dataframe(filter_condition='span_kind=="CHAIN"')
print(len(df))

5
1


In [27]:
df = px.Client().get_trace_dataset() ## project_name으로 구분

print(df)

In [ ]:
# Re-launch the app using the data

# px.launch_app(trace=px.TraceDataset(df))